In [1]:
import polars as pl
import altair as alt

In [2]:
pl.Config.set_tbl_rows(150)
pl.Config.set_fmt_str_lengths(150)
pl.Config.set_tbl_width_chars(200)

polars.config.Config

In [5]:
df = pl.scan_parquet(
    "data/projevy.parquet"
).filter(
    pl.col('datum').dt.year() >= 1918
).filter(
    ~(pl.col("komora_komplet").str.contains("Senát") 
      | 
      pl.col('komora_komplet').str.contains('Společn') 
      | 
      pl.col('komora').str.contains('psse')
     )
).drop_nulls(
    subset=['datum']
).with_columns(
    pl.col("text").str.count_matches(" ").alias("pocet_slov")
).sort(
    by='datum'
).join(
    pl.scan_parquet(
        "data/projevy_jazyky.parquet",
    ),
    how="left",
    on="text"
).filter(
    pl.col("jazyk") != "sk"
).collect()

In [6]:
def vyvoj_vyskytu(hledani, popisek):
    
    return df.with_columns(
        pl.col("text").str.count_matches(hledani).alias("vyskytu")
    ).group_by_dynamic(
        index_column='datum',every='1y'
    ).agg(
        pl.col("vyskytu").sum()
    ).join(
    df.group_by_dynamic(
        index_column='datum',every='1y'
    ).agg(
        pl.col("pocet_slov").sum()
    ),
    how="left",
    on="datum"
).with_columns(
    (pl.col("vyskytu") / pl.col("pocet_slov")).alias("pomer")
).with_columns(
    pl.lit(popisek).alias("co")
)

In [9]:
def graf(frejm):
    return alt.Chart(
    frejm.to_pandas(),
    width=800,
    height=100
).mark_bar(
).encode(
    alt.X("datum:T"),
    alt.Y("pomer:Q"),
    alt.Row("co:N")
)
    

In [11]:
import re

def hledej(text, n=3, delka=200, frejm=df):
    nalezy = frejm.sort(by="datum").filter(pl.col("text").str.contains(text)).head(n).iter_rows()
    for cislo, radek in enumerate(nalezy):
        print(f'Kdo: {radek[0]}')
        print(f'Kde: {radek[1]}')
        print(f'Kdy: {radek[9]}')
        
        # Split the text around the search term
        parts = re.split(text, radek[3])
        
        # Get the part before the search term (safe if empty)
        before = parts[0] if len(parts) > 0 else ""
        # Get the part after the search term (safe if empty)
        after = parts[1] if len(parts) > 1 else ""
        
        # Take up to 200 characters from before and after, but don't break if shorter
        before_snippet = before[-delka:] if len(before) > delka else before
        after_snippet = after[:delka] if len(after) > delka else after
        nalez = re.search(text, radek[3]).group()
        
        uryvek = before_snippet + nalez + after_snippet
        print(f'Co: {uryvek}')
        
        if cislo != 2:  # Changed from 3 to 2 since enumerate starts at 0
            print("***")

In [15]:
graf(vyvoj_vyskytu("(?i) sním o","sním o"))

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [17]:
graf(vyvoj_vyskytu("(?i)noční můr","noční můra"))

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [19]:
graf(vyvoj_vyskytu("(?i)vlhk\w{0,5} (sen|sny)","vlhký sen"))

<>:1: SyntaxWarning: invalid escape sequence '\w'
<>:1: SyntaxWarning: invalid escape sequence '\w'
C:\Users\micha\AppData\Local\Temp\ipykernel_32516\242411363.py:1: SyntaxWarning: invalid escape sequence '\w'
  graf(vyvoj_vyskytu("(?i)vlhk\w{0,5} (sen|sny)","vlhký sen"))
C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [21]:
graf(vyvoj_vyskytu("(?i)zdálo se mi o ","zdálo se mi o"))

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [25]:
graf(vyvoj_vyskytu("(?i) ve snu","ve snu"))

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [27]:
graf(vyvoj_vyskytu("(?i) snít o ","snít o"))

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [29]:
graf(vyvoj_vyskytu("(?i) (sen|sny) o ","sen/sny o"))

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [31]:
graf(vyvoj_vyskytu("(?i)halucin","halucinace"))

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [35]:
graf(vyvoj_vyskytu("(?i)chim[eé]r","chiméra"))

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [37]:
graf(vyvoj_vyskytu("(?i)přelud","přelud"))

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)

In [39]:
graf(vyvoj_vyskytu("(?i) vidin[ayoě]","vidiny"))

C:\Users\micha\anaconda3\Lib\site-packages\altair\utils\core.py:395: FutureWarning: the convert_dtype parameter is deprecated and will be removed in a future version.  Do ``ser.astype(object).apply()`` instead if you want ``convert_dtype=False``.
  col = df[col_name].apply(to_list_if_array, convert_dtype=False)


alt.Chart(...)